In [4]:
#pip install opensearch-py==2.5.0

In [6]:
from opensearchpy import helpers, OpenSearch

In [7]:
stg_vpce_opensearch = "aos-ef399bbbead7-qs5jilysrng2oxiiayoxwtklki.ap-northeast-2.es.amazonaws.com"
PORT = 443

In [8]:
client = OpenSearch(
    hosts = [{"host": stg_vpce_opensearch, "port": PORT}],
    http_compress=True,
    use_ssl=True,
    verify_certs=True,
    timeout=2,
    pool_maxsize=40,
    http_auth=("TheMasterUser", "Developer_rec_stg1!"),
)

In [9]:
index_name = 'onemodel_v3_inital_test_20240512'

# test 

In [14]:
from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)
spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/24 18:08:38 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [15]:
test_query_dict = spark.read.parquet('/data/temp/one_model_2024-05-11/new_query_240524.parquet.gzip')

In [19]:
test_embedding = test_query_dict.toPandas()

In [24]:
test_dict = test_embedding.set_index('new_query')['emb'].to_dict()

In [25]:
for query, emb in test_dict.items():
    query = query
    emb = emb 
    break

In [27]:
source = ["svc_mgmt_num", "mno_profiles", "adot_profiles"]

In [12]:
from typing import List, Dict
def vector_get_base_query(
        vector,
        source: List = [], 
        W: Dict = {},
    ):
    size = W.get("size", 5)
    base_query_dsl = {
        "_source": source,
        "size": size,
        "query": {
            "knn": {
                "user_vector": {
                    "vector": vector,
                    "k": size
                }
            }
        }
    }
    return base_query_dsl

In [28]:
vector_query = vector_get_base_query(vector=emb, source=source)

In [29]:
response = client.search(
    body = vector_query,
    index = index_name
)

RequestError: RequestError(400, 'search_phase_execution_exception', "failed to create query: Field 'user_vector' is not knn_vector type.")

In [35]:
response

{'took': 85,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 648, 'relation': 'eq'},
  'max_score': 3.9907799,
  'hits': [{'_index': 'onemodel_v3_inital_test_20240512',
    '_id': 'e209b6f0a9459257fe607e9db08d9fc2fb526ca00af18783660b947c6132e2b5',
    '_score': 3.9907799,
    '_source': {'svc_mgmt_num': 'e209b6f0a9459257fe607e9db08d9fc2fb526ca00af18783660b947c6132e2b5',
     'mno_profiles': '관심사:라이프스타일/취미_소셜 미디어,차량/운송_콜택시,차량/운송_차 구매<|n|>성별:남자<|n|>나이:47세<|n|>서비스 사용 기간:11년<|n|>기변 후 경과일:266일<|n|>요금제 이름:5GX 레귤러<|n|>요금제 가격:69000원<|n|>3개월 평균 데이터 사용량:9gb<|n|>단말기 가격:고가<|n|>단말기 제조사:samsung<|n|>멤버십 등급:vip<|n|>멤버십 사용 이력:없음<|n|>가족 결합 이력:있음<|n|>로밍 사용 이력:없음<|n|>세컨디바이스 보유 여부:없음<|n|>소액 및 DCB 결제 이력:0원',
     'adot_profiles': '선호 도메인 : 없음<|n|>선호 카테고리 : 없음<|n|>선호 아이템 : 없음<|n|>성별 : 남성<|n|>나이 : 47<|n|>활성 상태 : 휴면<|n|>재방문 비율 : 없음<|n|>다중 도메인 성향 : 없음<|n|>인기 컨텐츠 선호도 : 없음<|n|>사용성 기준 : 라이트유저<|n|>헤비유즈 도메인 : 없음'}},
   {'_index': 'onemodel_v3_in

In [ ]:
s